In [ ]:
from mqt.bench import get_benchmark
from qiskit import QuantumCircuit, Aer, execute, assemble
from qiskit.providers.fake_provider import FakeSherbrooke
from qiskit.compiler import transpile
from qiskit.visualization import plot_gate_map
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import numpy as np
import time
import os
from statistics import mean
import matplotlib.pyplot as plt
import re
import csv
from collections import OrderedDict
import warnings

# This bypasses the runtime warnings in the terminal
warnings.filterwarnings("ignore")
um = re.compile(r'(\d+)')
def ascending_sort(val):
    split = num.split(val)
    split[1::2] = map(int, split[1::2])
    return split

""" Path to the circuits stored locally

    Parameters
    ----------
    file_path : int
    
    Returns
    -------
    split
        x
"""  
def file_reader(file_path):
    circuits = []
    file_order = []
    directory = file_path
    for circuit in sorted(os.listdir(directory), key=ascending_sort):
        circuit_path = f"{file_path}/{circuit}"
        if(circuit_path.endswith('.qasm')):
            print(circuit_path)
            qc = QuantumCircuit.from_qasm_file(circuit_path)
            # Gets the name of each circuit in the order it's read
            file_order.append(circuit)
            circuits.append(qc)
    return circuits, file_order


def runtime_benchmarking(NUM_ITERATIONS, circuits, the_backend):
    
    BACKEND = the_backend  
    Optimization_Levels = {3: [], 2: [], 1: [], 0: []}
    
    # optimization_level_1 = []
    # optimization_level_2 = []
    # optimization_level_3 = []

    #These array will store:   
    mean_transpile_times_1 = [] 
    mean_transpile_times_2 = []
    mean_transpile_times_3 = []

    ttime_level1 = [] 
    ttime_level2 = []
    ttime_level3 = []
    
    counter = 0
    for circuit in circuits:
        
        transpiled = False
        temp1 = []
        temp2 = []
        temp3 = []     
        
            
        for _ in range(NUM_ITERATIONS):
                
            #Transpilation Level 1:
            start_time = time.perf_counter()
            qc1 = transpile(circuit, optimization_level= 1, seed_transpiler= 42, backend=BACKEND)
            stop_time = time.perf_counter()
            temp1.append(stop_time - start_time) 

            
            #Transpilation Level 2:
            start_time = time.perf_counter()
            qc2 = transpile(circuit, optimization_level= 2, seed_transpiler= 42, backend=BACKEND)
            stop_time = time.perf_counter()
            temp2.append(stop_time - start_time) 

                
            #Transpilation Level 3:
            start_time = time.perf_counter()
            qc3 = transpile(circuit, optimization_level= 3, seed_transpiler= 42, backend=BACKEND)
            stop_time = time.perf_counter()
            temp3.append(stop_time - start_time) 

            
            #If this is the first iteration, then we simply add the circuits to the dictonary
            if transpiled == False:
                
                Optimization_Levels[3].append(qc3)
                Optimization_Levels[2].append(qc2)
                Optimization_Levels[1].append(qc1)
                Optimization_Levels[0].append(circuit)
                transpiled = True
                
        #At this point all the data has been added to iteration_times. Now it is just a matter of extracting data. 
        ttime_level1.append(temp1)
        ttime_level2.append(temp2)
        ttime_level3.append(temp3)


        mean_transpile_times_1.append(mean(temp1))
        mean_transpile_times_2.append(mean(temp2))
        mean_transpile_times_3.append(mean(temp3))
        
        print("Circuit Index Completed: ", counter)
        counter += 1
            
    #Scatter Plot for Runtime after all values are collected
    plt.figure(figsize=(12, 6))

    #Number of qubits in the sorted circuit
    number_of_qubits = [i + 2 for i in range(len(mean_transpile_times_1))] #num of qubits 
    
    x = np.array(number_of_qubits)

    #Calculating Line of BEST FIT: Optimization Level 1
    a, b = np.polyfit(x, np.array(mean_transpile_times_1), 1)
    #Calculating Line of BEST FIT: Optimization Level 2
    c, d = np.polyfit(x, np.array(mean_transpile_times_2), 1)
    #Calculating Line of BEST Fit: Optimization Level 3
    e, f = np.polyfit(x, np.array(mean_transpile_times_3), 1)


    plt.scatter(number_of_qubits, mean_transpile_times_1, label = "Average of Opt Level 1")
    plt.plot(x, a*x+b)
    plt.scatter(number_of_qubits, mean_transpile_times_2, label = "Average of Opt Level 2")
    plt.plot(x, c*x+d)
    plt.scatter(number_of_qubits, mean_transpile_times_3, label = "Average of Opt Level 3")
    plt.plot(x, e*x+f)
    plt.xlabel('Number of Qubits')
    plt.ylabel('Runtime in Seconds')
    plt.title('Runtime in Seconds (at each opt level)')
    plt.legend()
    plt.show()

    print("mean 1", mean_transpile_times_1)
    print("mean 2", mean_transpile_times_2)
    print("mean 3", mean_transpile_times_3)
    
    #return mean_transpile_times_1
    return Optimization_Levels, ttime_level1, ttime_level2, ttime_level3
    #return optimization_level_1, optimization_level_2, optimization_level_3




In [ ]:
backend = FakeSherbrooke()
circuits, file_order = file_reader("tests") # have to change folder directory for the circuits
transpiled_circuits = runtime_benchmarking(5, circuits, backend)

# Retrieves the return values from the benchmarking methods for the CSV file
#transpile = runtime_benchmarking(5, circuits, backend)
gate_count_lvl_1, gate_count_lvl_2, gate_count_lvl_3 = gate_count(transpiled_circuits)
qubit_ratio_lvl_1, qubit_ratio_lvl_2, qubit_ratio_lvl_3 = single_multi_ratio_benchmarking(transpiled_circuits)

""" 
    Here we are creating the CSV file to store the results from our benchmarks.
    The data we are collecting includes:
    The name of the circuit 
    The runtime
    The gate count
    The ratio of single to multi qubit gates
    The of number of swap gates
    The number of entangled gates
""" 
with open('test_circuits_opt_1.csv', 'w', newline='') as csvfile:
    # Below is the information we are trying to extract from our circuits
    fieldnames = ['circuit name',' average runtime',' runtime level 1', ' runtime level 2' , ' runtime level 3',
                  ' runtime level 4', ' runtime level 5' ,' gate count', ' single to multi qubit ratio']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames) # calling the writer
    writer.writeheader() # Here we are creating the columns for our CSV file

    for file_name, gate, ratio in zip(file_order, gate_count_lvl_1, qubit_ratio_lvl_1):
        writer.writerow({'circuit name': file_name,
                         ' gate count': gate,
                         ' single to multi qubit ratio': ratio})
                        